In [244]:
from langchain import OpenAI
import os
from typing import Dict, TypedDict, Optional, Annotated, Union
from langchain.prompts import PromptTemplate
import re
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI, AzureChatOpenAI
import openai
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from dotenv import load_dotenv
from langchain_core.pydantic_v1 import BaseModel, Field

# Agent

from langchain.tools import BaseTool
from langchain.agents import AgentExecutor
import langchain

from langchain.agents import AgentExecutor, create_react_agent, Tool


from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

from langchain_core.runnables import RunnableLambda
import json
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from operator import itemgetter


In [245]:
# Debug langchain logs, True if yes else set false
langchain.debug = False

## Retrieval Component

In [251]:

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('para_olympics.txt').load()
# choose chunk size and chunk overlap
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
documents = text_splitter.split_documents(raw_documents)

In [252]:
# check the splitted docs
print("No. of chunks created", len(documents))

No. of chunks created 4


In [253]:
load_dotenv()

True

In [254]:
OPENAI_KEY=os.getenv("OPENAI_KEY")
OPENAI_URL=os.getenv("OPENAI_URL")
openai_api_version=os.getenv("openai_api_version")
azure_deployment=os.getenv("azure_deployment")

In [255]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import numpy as np
embedding = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    chunk_size=1,
    api_key =OPENAI_KEY,
    azure_endpoint =OPENAI_URL
)

In [256]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embedding)

In [257]:
# view vector store
vectorstore.docstore._dict

{'3629fc06-2963-4c74-a412-5b4a30fc72fc': Document(metadata={'source': 'para_olympics.txt'}, page_content='## Introduction\n\nThe Paris 2024 Paralympic Games, scheduled to take place from August 28 to September 8, 2024, will mark a significant milestone in the history of the Paralympic movement. As the first Paralympic Games to be held in Paris, this event is expected to showcase the extraordinary talents of over 4,000 athletes from around the globe, competing in 549 medal events across 22 sports. This essay explores the significance of the 2024 Paralympics, the sports involved, the athletes to watch, the event\'s historical context, and its impact on society.\n\n## Historical Context of the Paralympic Games\n\nThe Paralympic Games have evolved significantly since their inception. The first official Paralympic Games were held in Rome in 1960, featuring 400 athletes from 23 countries. This event was rooted in the Stoke Mandeville Games, organized by Dr. Ludwig Guttmann in 1948 to aid the

## Augumented Component

In [258]:
# retriever setup
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7, "k":1}
) | format_docs

In [259]:
docs = retriever.invoke("when it hosted this game?")

/Users/shyamsi1-dhl/workspace/projects/AI/GEN-AI/lang-chain-project/venv/lib/python3.12/site-packages/langchain_core/vectorstores/base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.7
  warnings.warn(


In [260]:
# format docs
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Generation Component

In [261]:
# k - no of latest conversation in memory
memory = ConversationBufferMemory(memory_key="chat_history",  k=5,return_messages=True)

In [262]:
# check if there is any message
memory.load_memory_variables({})

{'chat_history': []}

In [263]:
# llm model instance
llm = AzureChatOpenAI(
        api_key=OPENAI_KEY,
        azure_endpoint=OPENAI_URL,
        openai_api_version=openai_api_version,
        azure_deployment=azure_deployment,
        temperature=0,
        verbose=True,
        model_kwargs={},
    )

In [264]:
# prompt
system_prompt = (
    """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer 
    the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n
    context:```{context}```
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)



In [265]:
retrieval = RunnableParallel(
    {"context": itemgetter("question")| retriever, "question": itemgetter("question") , "chat_history": itemgetter("chat_history")}
)

rag_chain = retrieval | prompt | llm  | StrOutputParser()

rag_chain.invoke({"question":"medal country wise in paraolympics", "chat_history":memory.load_memory_variables({})['chat_history']})

'The medal distribution by country for the Paris 2024 Paralympics is as follows:\n1. China: 94 Gold, 76 Silver, 50 Bronze, Total 220\n2. Great Britain: 49 Gold, 44 Silver, 31 Bronze, Total 124\n3. United States: 36 Gold, 42 Silver, 27 Bronze, Total 105\n4. Netherlands: 27 Gold, 17 Silver, 12 Bronze, Total 56\n5. Neutral Paralympic Athletes: 26 Gold, 22 Silver, 23 Bronze, Total 71\n6. Brazil: 25 Gold, 26 Silver, 38 Bronze, Total 89\n7. Italy: 24 Gold, 15 Silver, 32 Bronze, Total 71\n8. Ukraine: 22 Gold, 28 Silver, 32 Bronze, Total 82\n9. France: 19 Gold, 28 Silver, 28 Bronze, Total 75\n10. Australia: 18 Gold, 17 Silver, 28 Bronze, Total 63\n11. Japan: 14 Gold, 10 Silver, 17 Bronze, Total 41\n12. Remaining NPCs (85): 195 Gold, 226 Silver, 289 Bronze, Total 710'

## Plain RAG Q&A App

In [266]:
user_input = ""
while user_input != "q":
    user_input = input("Enter text for Para Olympic Q&A with memory  (press 'q' to quit): ")
    if user_input == "q":
        print('Exiting the app')
        # clear the buffer memory
        memory.clear()
        break
    memory_result = rag_chain.invoke({"question":user_input, "chat_history":memory.load_memory_variables({})['chat_history']})
    print("AI:", memory_result)

    memory.chat_memory.add_user_message(user_input)
    memory.chat_memory.add_ai_message(memory_result)

Enter text for Para Olympic Q&A with memory  (press 'q' to quit):  q


Exiting the app


# Issue with RAG
### 1 : Rephrasing questions
Not all questions can be found in docs

In [ ]:
# - medal tally for 2024 para olympics
# who won maximum medal?
# when it hosted this game?
# can you summarise in bullet points
# can you repeat the last question
# sorry, i did not get it

### Try fixing this issue? - Rephrase question

In [270]:
### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{question}"),
    ]
)

rephrase_chain = contextualize_q_prompt | llm | StrOutputParser()

In [271]:
retrieval = RunnableParallel(
    {"context": rephrase_chain | retriever, "question": itemgetter("question") , "chat_history": itemgetter("chat_history")}
)

rag_chain_rephrase = retrieval | prompt | llm  | StrOutputParser() 

In [272]:
rag_chain_rephrase.invoke({"question":"who won maximum medal?", "chat_history":memory.load_memory_variables({})['chat_history']})

'China won the maximum number of medals at the Paris 2024 Paralympics, with a total of 220 medals, including 94 gold, 76 silver, and 50 bronze.'

In [ ]:
## issue after rephrasing, documents found but model did not account complete context before answering it.

### 2: Issue with tight coupling of RAG components

# Agents

## Create tools

In [273]:
# get year of hosting and place of hosting by country name
def get_paralympic_game(country):
    with open('para_olympics.json') as file:
        data = json.load(file)
    
    print('data', data)
    for game in data['games']:
        if game['country'] == country:
            return game
    
    return None

In [274]:
# get medal tally by country name
def get_medal_tally_by_country(country):
    with open('medal.json') as file:
        data = json.load(file)
    
    for row in data['medal_tally']:
        if row['country'].lower() == country.lower():
            return row
    return None

### retriever tool

In [275]:
# tools

class SearchInput(BaseModel):
    question: str = Field(description="Rephrased Question from the given chat history")


class RetrieverTool(BaseTool):
    name = "retriever_tool"
    description = "This tool is useful to answer question related to Para olympics history, signficance, sport highlights, paris 2024 paraolympics event etc"
    args_schema: Type[BaseModel] = SearchInput

    def _run(
        self, question: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return retriever.invoke(question)
    

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("document_retriever_tool does not support async")

### get country hosting

In [276]:
# tools

class Country(BaseModel):
    country: str = Field(description="Name of the country")


class CountryHosting(BaseTool):
    name = "country_hosting"
    description = "This tool is useful to answer questions related to country hosted para olympics games. It provide list of country which has hosted para olympics game "
    args_schema: Type[BaseModel] = Country

    def _run(
        self, country: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        result = get_paralympic_game(country)
        if result:
            return str(result)
        else:
            "No information found"
    

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("document_retriever_tool does not support async")

### get country by medal

In [277]:
# tools

class Country(BaseModel):
    country: str = Field(description="Name of the country")


class CountryMedalTally(BaseTool):
    name = "country_medal_tally"
    description = "This tool is useful to answer questions related to para olympics medal tally. It provides information about country wining medals in paris para olympics games in 2024 "
    args_schema: Type[BaseModel] = Country

    def _run(
        self, country: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        result = get_medal_tally_by_country(country)
        if result:
            return str(result)
        else:
            "No information found"
    

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("document_retriever_tool does not support async")

### list of tools available for agent to take action

In [278]:
tools =[RetrieverTool(), CountryHosting(), CountryMedalTally()]

### agent configuration

In [279]:
template = '''Answer the following questions as best you can. You have access to the following tools:
            {tools}

            Use the following format:

            Question: the input question you must answer
            Thought: you should always think about what to do
            Action: the action to take, should be one of [{tool_names}]
            Action Input: the input to the action
            Observation: the result of the action
            ... (this Thought/Action/Action Input/Observation can repeat N times)
            Thought: I now know the final answer
            Final Answer: the final answer to the original input question

            You need to first look to the conversation history if the question can be answered with 100% confidence or take refrence of conversation history to analyse/rephrase question and then use proper tools to answer the question

            Begin!

            Conversation history: {chat_history}

            Question: {input}
            Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [280]:
# ReAct Agent
react_agent = create_react_agent(llm, tools, prompt)

In [281]:
agent_executor = AgentExecutor(agent=react_agent, tools=tools, verbose=True, memory=memory, early_stopping_method='force',
                                       handle_parsing_errors=True, max_iterations=3)

## Agent based Q&A App 

In [282]:
user_input = ""
while user_input != "q":
    user_input = input("Enter question for Para Olympics 2024  (press 'q' to quit): ")
    if user_input == "q":
        memory.clear()
        print('Exiting the app')
        break
    agent_result = agent_executor.invoke({"input":user_input, "chat_history":memory.load_memory_variables({})['chat_history']})
    print("\n==============Result======================\n", agent_result['output'])

Enter question for Para Olympics 2024  (press 'q' to quit):  q


Exiting the app


In [205]:

# how many gold medal china won?
# when it hosted this game?
# history of this game?
# can you summarise the question in three bullet points
# can you repeat the last question

# show agent thought and scratch pad and how its repharsing question automatically to find something